# Hyperparameter
1. **Learned parameters, weights and biases, are learned during training**.
* For neural networks, these typically are the weights on each neural network connection, and the biases on each node.
* For CNNs, learned parameters are the filters in each convolutional layer.

These learned parameters stay as part of the model when the model is done training

Hyperparameters are parameters used to train the model, but are not part of the trained model itself. After training, the hyperparameters no longer exist. Hyperparameters are used to improve the training of the model, by answering questions such as these:
1. How long does it take to train the model?
2. How fast does the model converge?
3. Does it find the global optimum?
4. How accurate is the model?
5. How overfitted is the model?


Another perspective of hyperparameters is that they are a means to measure the cost and quality of developing the model.

## Epochs
The most basic hyperparameter is the number of epochs, though this is now being more commonly replaced with steps. The epochs hyperparameter is the number of times you will pass the entire training data through the neural network during training.

Training is very expensive in terms of computing time. It includes both the forward feeding to pass the training data through and the backward propagation to
update (train) the model’s parameters. For example, if a full pass of the data (epoch) takes 15 minutes, and we run 100 epochs, the training time will take 25 hours.

## Steps
Another way to improve accuracy and reduce training time is by changing the sampling distribution of the training dataset. For epochs, we think of a sequential draw of batches from our training data. Even though we randomly shuffle the training data at the start of each epoch, the sampling distribution is still the same

Let’s now think of the entire population of the subject we want to recognize. In statistics, we call this the population distribution:

<img src="img_12.png" />

But we will never have a dataset that is the actual entire population distribution. Instead, we have samples, which we refer to as a sampling distribution of the population distribution.

 Another way to improve our model is to additionally learn the best sampling distribution for training the model. Although our dataset may be fixed, we can use several techniques to alter the distribution, and thus learn the sampling distribution that best fits training the model. These methods include the following:
 1. Regularization / Dropout
 2. Batch Normalization
 3. Data Augmentation

From this perspective, we no longer see feeding the neural network as sequential passes over the training data, but as making random draws from a sampling distribution. In this context, steps refers to the number of batches (draws) we will make from the sampling distribution of our training data.

we no longer see feeding the neural network as sequential passes over the training data, but as making random draws from a sampling distribution. In this context, steps refers to the number of batches (draws) we will make from the sampling distribution of our training data.

<img src="img_13.png" />

When we add dropout layers to the neural networks, we are randomly dropping activations on a per-sample basis. In addition to reducing overfitting of a neural network, we are also changing the distribution

With batch normalization, we are minimizing covariance shift between our batches of training data (samples). Just as we use standardization on our input, the activations are rescaled using standardization (we subtract the batch mean and divide by the batch standard deviation). This normalization reduces the fluctuations in updates to parameters in the model; this process is referred to as adding more stability to the training. In addition, this normalization mimics drawing from a sampling distribution that is more representative of the population distribution.

With data augmentation (discussed in chapter 13), we create new examples by modifying existing examples within a set of parameters. We then randomly select the
modification, which also contributes to changing the distribution.

With batch normalization, regularization/dropout, and data augmentation, no two epochs will have the same sampling distribution.In this case, the practice now is to limit the number of random draws (steps) from each new sampling distribution, further changing the distribution.

 For example, if steps are set to 1000, then per epoch, only 1000 random batches will be selected and fed into the neural network for training.

In TF.Keras, we can specify both the number of epochs and steps as parameters to
the fit() method, as the parameters epochs and steps_per_epoch:


In [2]:
from keras import Sequential
from keras.layers import Dense
import numpy as np
import tensorflow as tf

In [9]:
X = np.random.random((28,28,3))
y = 2*X**2 +0.5 * X +10
batch_size = 2
epochs = 100
model = Sequential([Dense(3, activation='relu', input_shape=(28,3))])
model.compile(loss='mse', optimizer='adam', metrics=[tf.keras.metrics.RootMeanSquaredError()])
model.fit(X, y, batch_size=batch_size, epochs=epochs, steps_per_epoch=1000)

Epoch 1/100
1000/1000 [==============================] - 1s 497us/step - loss: 85.2106 - root_mean_squared_error: 9.2310
Epoch 2/100
1000/1000 [==============================] - 0s 238us/step - loss: 58.2868 - root_mean_squared_error: 7.6346


## Batch Sizes
To understand how to set batch size, you should have a basic understanding of the three types of gradient descent algorithms:
1. stochastic gradient descent,
2. batch gradient descent, and
3. mini-batch gradient descent.

The algorithm is the means by which the model parameters are updated (learned) during training.


### STOCHASTIC GRADIENT DESCENT
In stochastic gradient descent (SGD), the model is updated after each example is fed through during training. Since each example is randomly selected, the variance between examples can result in large swings in the gradient.

A benefit is that during training, we are less likely to converge on a local (that is, lessor) optimum, and more likely to find the global optimum to converge on. Another benefit is that the rate of change in loss can be monitored in real time, which may aid in algorithms that do automatic hyperparameter tuning. The downside is that this is more computationally expensive per epoch

### BATCH GRADIENT DESCENT
In batch gradient descent, the error loss per example is calculated as each example is fed through during training, but the updating of the model is done at the end of each epoch (after the entire training data is passed through). As a result, the gradient is smoothed out because it’s calculated across the loss of all the examples, instead of a single example.


 The benefits are that this is less computationally expensive per epoch, and the training more reliably converges. The downsides are that the model may converge on a less accurate local optimum, and an entire epoch needs to be run to monitor performance data.

### MINI-BATCH GRADIENT DESCENT
The mini-batch gradient descent method is a tradeoff between stochastic and batch gradient descent. Instead of one example or all examples, the neural network is fed in mini-batches that are a subset of the entire training data. The smaller the mini-batch side, the more the training will resemble stochastic gradient descent, while larger batch sizes will resemble batch gradient descent.

 For certain models and datasets, SGD works best. In general, it’s a common practice to use the tradeoff of mini-batch gradient descent. The hyperparameter batch_size indicates the size of the mini-batch. Because of hardware architectures, the most time/space-efficient batch sizes are multiples of 8, such as 8, 16, 32, and 64. The batch size that is most commonly tried first is 32, and then 128. For extremely large datasets on higher-end hardware (HW) accelerators (such as GPUs and TPUs), it is common to see batch sizes of 256 and 512. In TF.Keras, you can specify batch_size in the model fit() method:
model.fit(x_train, y_train, batch_size=32)


## Learning rate
The learning rate is generally the most influential of the hyperparameters. It can have a significant impact on the length of time to train a neural network as well as on whether the neural network converges on a local (lessor) optimum, and whether it converges on the global (best) optimum.

The learning rate provides us with a means to control the degree that the model parameters are updated. In the basic method, the learning rate is a fixed coefficient between 0 and 1 that is multiplied against the value to add/subtract, to reduce the amount being added or subtracted. These smaller increments add more stability during the training and increase the likelihood of convergence.

### Small vs Large Learning Rate

If we use a very small learning rate, like 0.001, we will eliminate large swings in the model parameters during updates. This will generally guarantee that the training will converge on a local optimum. But there is a drawback.
1. First, the smaller we make the increments, the more passes of the training data (epochs) will be needed to minimize the loss. That means more time to train.
2. Second, the smaller the increments, the less likely the training will explore other local optima, which might be more accurate than the one that the training is converging on; instead, it may converge on poor local optimum or get stuck on a saddle point.

A large learning rate, like 0.1, likely will cause big jumps in the model parameters during updates. In some cases, it might initially lead to faster convergence (fewer epochs). The drawback is that even if you are initially converging fast, the jumps may overshoot and start causing the convergence to swing back and forth, or hop across different local optima. At very high learning rates, the training may start to diverge (increasing loss).

### Decay

One common practice has been to start with a slightly larger learning rate, and then gradually decrease it, also referred to as learning rate decay. The larger learning rate would at first explore different local optima to converge on and make initial deep swings into the respective local optima. The rate of convergence and minimizing the loss function on the initial updates can be used to home in on the best (good) local optimum.

From that point, the learning rate is gradually decayed. As the learning rate decays, it is less likely for swings out of the good local optimum to occur, and the steadily decreasing learning rate will tune the convergence to approach the minimal point (albeit, the smaller and smaller learning rate will increase training time). So the decay becomes a tradeoff between small increases in final accuracy and the overall training time.  The following is a basic formula adding decay to the calculation of updating the weights. On each update, the learning rate is reduced by the decay amount (called the fixed decay):
>>  weight += -learning_rate * gradient
    learning_rate -= decay


In practice, the decay formulas are generally time-based, step-based, or cosine-based decays. These formulas can be expressed in simplified terms, and an iteration may be a batch or epoch. By default, TF.Keras optimizers use time-based decay. The formulas are as follows:
 Time-based decay
>> learning_rate *= (1 / (1 + decay * iteration))


 Step-based decay
>> learning_rate = initial_learning_rate * decay**iteration


 Cosine decay
>>learning_rate = c * (1 + cos(pi * (steps_per_epoch * interaction)/epochs))
where c is typically in range 0.45 to 0.55


### MOMENTUM
Another common practice is to accelerate or decelerate the rate of change based on prior changes. If we have large jumps in convergence, we risk jumping out of the local optimum, so we may want to decelerate the learning rate. If we have small to no changes in convergence, we may want to accelerate the learning rate to hop over a saddle point. Typically, values for momentum range from 0.5 to 0.99:


>>  velocity = (momentum * velocity) - (learning_rate * gradient)
    weight += velocity

### ADAPTIVE LEARNING RATE
Many popular algorithms dynamically adapt the learning rate:
 Adadelta
 Adagrad
 Adam
 AdaMax
 AMSGrad
 Momentum
 Nadam
 Nesterov
 RMSprop